In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import sparse_dot_topn.sparse_dot_topn as ct
from scipy.sparse import csr_matrix
from sparse_dot_topn import awesome_cossim_topn


In [2]:
trademark = pd.read_csv("Trademark data compiled.csv")
sec = pd.read_csv("sec_list.csv")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,9,12,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def ngrams(string, n=3):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [4]:
def get_matches_df(sparse_matrix, A, B, top=100):
    non_zeros = sparse_matrix.nonzero()

    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]

    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size

    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)

    for index in range(0, nr_matches):
        left_side[index] = A[sparserows[index]]
        right_side[index] = B[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]

    return pd.DataFrame({'left_side': left_side,
                         'right_side': right_side,
                         'similairity': similairity})

In [5]:
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix_clean = vectorizer.fit_transform(sec['companyName'])
tf_idf_matrix_dirty = vectorizer.transform(trademark['own_name'])

In [7]:
t1 = time.time()
matches = awesome_cossim_topn(tf_idf_matrix_dirty, tf_idf_matrix_clean.transpose(), 1, 0)
t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 282.04255390167236


In [8]:
matches_df = get_matches_df(matches, trademark['own_name'], sec['companyName'], top=0)

In [9]:
matches_df.to_csv('Matched using TDIDF.csv')

In [26]:
matches_df.loc[matches_df['similairity'] > 0.7].drop_duplicates().sort_values(by = ['similairity'], ascending = True).head(300)

,left_side,right_side,similairity
2591088,Sterling Life Insurance Company,Pruco Life Insurance Company,0.700018
1371899,"Air Technologies, Inc.","Uber Technologies, Inc",0.700021
2172054,LSI CORPORATION,LSI CORP,0.700041
711573,"CHAMPPS ENTERTAINMENT, INC.","T.O ENTERTAINMENT, INC.",0.700042
159130,"LUFT INSTRUMENTS, INC.",BISON INSTRUMENTS INC,0.700051
3640521,KMT Enterprises LLC,"Lee Enterprises, Inc",0.700056
745009,ROI DEVELOPMENT CORP.,MI DEVELOPMENTS INC,0.700059
3039166,9278 COMMUNICATIONS INC.,B COMMUNICATIONS LTD,0.700073
4364024,"O.K. FOODS, INC.","US FOODS, INC.",0.700078
4346811,Volcano Grill Inc,Volcano Corp,0.700082


In [10]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(matches_df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

